In [1]:
import numpy as np
import pandas as pd

In [2]:
movies=pd.read_csv("tmdb_5000_movies.csv")
credits=pd.read_csv("tmdb_5000_credits.csv")

In [3]:
movies=movies.merge(credits,on="title")

In [4]:
#cleaning
#important
#genres
#id
#keywords
#title
#overview
#cast
#crew
movies=movies[["movie_id","title","overview","genres","keywords","cast","crew"]]

In [5]:
#data preprocessing
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [6]:
movies.dropna(inplace=True)

In [7]:
movies.duplicated().sum()

0

In [8]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [9]:
import ast
def convert(obj):
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

In [10]:
movies['genres']=movies['genres'].apply(convert)

In [11]:
movies['keywords']=movies['keywords'].apply(convert)

In [12]:
def convert3(obj):
    l=[]
    count=0
    for i in ast.literal_eval(obj):
        if count!=3:
            l.append(i['name'])
            count+=1
    return l

In [13]:
movies['cast']=movies['cast'].apply(convert3)

In [14]:
def fetch_director(obj):
    l=[]
    for i in ast.literal_eval(obj):
        if(i['job']=='Director'):
            l.append(i['name'])
            break
    return l

In [15]:
movies['crew']=movies['crew'].apply(fetch_director)

In [16]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [17]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","")for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","")for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","")for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","")for i in x])

In [18]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [19]:
new_df=movies[['movie_id','title','tags']]

In [20]:
new_df['tags']=new_df['tags'].apply(lambda x:(" ").join(x))

C:\Users\Dhananjay\AppData\Local\Temp/ipykernel_15712/3705085487.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:(" ").join(x))


In [21]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

C:\Users\Dhananjay\AppData\Local\Temp/ipykernel_15712/4224080999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower())


In [22]:
import nltk

In [23]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [24]:
def stem(text):
    y=[]
    
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [25]:
new_df['tags']=new_df['tags'].apply(stem)

C:\Users\Dhananjay\AppData\Local\Temp/ipykernel_15712/3514595201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


In [26]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [27]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [28]:
from sklearn.metrics.pairwise import cosine_similarity

In [29]:
similarity=cosine_similarity(vectors)

In [30]:
def recommend(movie):
    movie_index=new_df[new_df['title']==movie].index[0]
    distances=similarity[movie_index]
    movie_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movie_list:
        print(new_df.iloc[i[0]].title)

In [31]:
import pickle

In [32]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [33]:
pickle.dump(similarity,open('similarity.pkl','wb'))